# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install citipy

In [2]:
import json
import os


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from scipy import stats

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count  (this is a list from citipy)
len(cities)

637

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [5]:
# Test the code using only 2 cities instead of 600+
#cities = ['Paris', 'London']

In [6]:
url = 'https://openweathermap.org/api'

query_url = url + 'appid=' + weather_api_key + '&q='


# Loop through the list of cities and perform a request for data on each

for x in range(len(cities)):
    print(f'Processing: {x} : {cities[x]}')



#api.openweathermap.org/data/2.5/weather?q={city name}&appid={API key}

#a = requests.get(url)
#response = a.json()
#print(response)

Processing: 0 : pangnirtung
Processing: 1 : hilo
Processing: 2 : fortuna
Processing: 3 : adrar
Processing: 4 : boguchany
Processing: 5 : villa del rosario
Processing: 6 : barrow
Processing: 7 : chadiza
Processing: 8 : gat
Processing: 9 : vitim
Processing: 10 : iqaluit
Processing: 11 : hobart
Processing: 12 : pochutla
Processing: 13 : sangar
Processing: 14 : sao filipe
Processing: 15 : evensk
Processing: 16 : nsanje
Processing: 17 : amderma
Processing: 18 : howard springs
Processing: 19 : natal
Processing: 20 : kapaa
Processing: 21 : upernavik
Processing: 22 : kracheh
Processing: 23 : naifaru
Processing: 24 : ushuaia
Processing: 25 : kodiak
Processing: 26 : cap malheureux
Processing: 27 : atuona
Processing: 28 : kropotkin
Processing: 29 : talawdi
Processing: 30 : marcona
Processing: 31 : lavrentiya
Processing: 32 : port alfred
Processing: 33 : hermanus
Processing: 34 : constitucion
Processing: 35 : kilrush
Processing: 36 : kununurra
Processing: 37 : aldan
Processing: 38 : souillac
Proce

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [7]:
# Values needed from api: city name, lat, lng, max temp, humidity, cloudiness, windspeed, country, date

name = []
lat = []
lng = []
max_temp = []
humid = []
cloud = []
wind = []
country = []
date = []

url_and_api = "http://api.openweathermap.org/data/2.5/weather?units=metric&appid="+ weather_api_key

celsius = '&units=metric'



# Study documentation to see how to make the request
for city in cities:
    
    try:
    
        response = requests.get(f'{url_and_api}&q={city}').json()
        
        name.append(response['name'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        
        max_temp.append(response['main']['temp_max'])
        humid.append(response['main']['humidity'])
        
        cloud.append(response['clouds']['all'])
        wind.append(response['wind']['speed'])
        country.append(response['sys']['country'])
        date.append(response['dt'])
        
    
    except:
        name.append(None)
        lat.append(None)
        lng.append(None)
        max_temp.append(None)
        humid.append(None)
        cloud.append(None)
        wind.append(None)
        country.append(None)
        date.append(None)
        
    continue


In [8]:
weather_data = pd.DataFrame({'City':cities, 
                             'Lat':lat,
                             'Lng':lng,
                             'Max Temp':max_temp,
                             'Humidity':humid,
                             'Cloudiness':cloud,
                             'Wind Speed':wind,
                             'Country':country,
                             'Date':date
                            })

weather_data.dropna(inplace=True)
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,pangnirtung,66.1451,-65.7125,-27.00,69.0,20.0,1.68,CA,1.615659e+09
1,hilo,19.7297,-155.0900,21.00,78.0,75.0,2.22,US,1.615659e+09
2,fortuna,40.5982,-124.1573,11.11,81.0,90.0,3.09,US,1.615659e+09
3,adrar,20.5022,-10.0711,27.35,9.0,81.0,3.75,MR,1.615659e+09
4,boguchany,58.3814,97.4531,-17.14,96.0,22.0,2.73,RU,1.615659e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [14]:
weather_data['Humidity'.max()


City                 zile
Lat               78.2186
Lng               178.417
Max Temp            38.08
Humidity              100
Cloudiness            100
Wind Speed          16.98
Country                ZM
Date          1.61566e+09
dtype: object

In [15]:
#  Get the indices of cities that have humidity over 100%.
print('There is not humidity over 100%')

There is not humidity over 100%


In [10]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression